# SpaceTime Playground

In [2]:
%reload_ext autoreload
%autoreload 3

import torch
import torch.nn.functional as F
from torchinfo import summary
import pytorch_lightning as pl
from src.b2bnet import B2BNetSpaceTimeModel, OtkaDataModule, OtkaTimeDimSplit
from pytorch_lightning.loggers import TensorBoardLogger

In [8]:
# Experiment

segment_size = 120 * 3  # 3sec
batch_size = 256
n_channels = 59
n_features = 8
hidden_size = 8
max_epochs = 100

datamodule = OtkaTimeDimSplit(segment_size=segment_size, batch_size=batch_size)

model = B2BNetSpaceTimeModel(
    n_channels=n_channels, n_features=n_features,
    hidden_size=hidden_size, kernel_size=1, n_subjects=51)

trainer = pl.Trainer(max_epochs=max_epochs,accelerator='cpu', log_every_n_steps=1, deterministic='warn')

trainer.fit(model, datamodule=datamodule)

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name        | Type                 | Params
-----------------------------------------------------
0 | autoencoder | SpaceTimeAutoEncoder | 2.2 K 
1 | cls         | Linear               | 18    
-----------------------------------------------------
2.2 K     Trainable params
0         Non-trainable params
2.2 K     Total params
0.009     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=100` reached.
